In [1]:
import pandas as pd

In [2]:
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS
from bokeh.models.widgets import DataTable, TableColumn, Select
from bokeh.plotting import save, output_file

In [3]:
df = pd.read_table("output/player_ranks.txt", index_col=0)
df.reset_index(inplace=True, drop=True)
df.index = df.index.values + 1

In [4]:
source = ColumnDataSource(df)
original_source = ColumnDataSource(df)

In [5]:
from bokeh.models.widgets import NumberFormatter

In [6]:
numform = NumberFormatter(format='0.00')

strcolumns = [TableColumn(field=x, title=x) for x in ["coach", "race"]]
numcolumns = [TableColumn(field=x,
                          title=x, 
                          formatter=numform) for x in ["mu", "phi", "value"]]

data_table = DataTable(source=source, columns=strcolumns + numcolumns)

In [7]:
df.head()

,coach,race,mu,phi,value
1,Jimjimany,Wood Elves,1884.210,62.254,1728.5750
2,strider84,Wood Elves,1868.620,59.565,1719.7075
3,Joemanji,Undead,1838.464,53.228,1705.3940
4,Pipey,Norse,1886.250,74.396,1700.2600
5,Jokaero,Dark Elves,1879.240,75.342,1690.8850


In [8]:
data_table.height = 1000
data_table.width = 500

In [9]:
# callback code to be used by all the filter widgets
# requires (source, original_source, race_select_obj,target_object)
combined_callback_code = """
var data = source.get('data');
var original_data = original_source.get('data');
var race = race_select_obj.get('value');
console.log("race: " + race);

for (var key in original_data) {
    data[key] = [];
    for (var i = 0; i < original_data['race'].length; ++i) {
        if (race === "ALL" || original_data['race'][i] === race) {
            data[key].push(original_data[key][i]);
        }
    }
}

target_obj.trigger('change');
source.trigger('change');

"""

In [10]:
# define the filter widgets, without callbacks for now
race_list = ['ALL'] + df['race'].unique().tolist()
race_select = Select(title="Race:", value=race_list[0], options=race_list)

# year_list = ['ALL'] + df['year'].unique().tolist()
# year_select = Select(title="Year:", value=year_list[0], options=year_list)
# now define the callback objects now that the filter widgets exist
generic_callback = CustomJS(
    args=dict(source=source, 
              original_source=original_source, 
              race_select_obj=race_select, 
              target_obj=data_table),
    code=combined_callback_code
)

# finally, connect the callbacks to the filter widgets
race_select.js_on_change('value', generic_callback)

#year_select.js_on_change('value', generic_callback)
p = column(race_select, data_table)

output_file('rankings.html')

save(p)

'/home/njh/git/NAF/datatable_filter.html'